Header Box: Title, names, etc.

### Monte Carlo scattering calculations

This model predicts the reflectance spectra in structurally colored samples made from nanoscale spheres in a disordered packing. Past models have accounted for only a single scattering event in the sample, but this model uses a Monte Carlo approach to account for many scattering events, leading to more accurate results. 

The Monte Carlo model simulates the trajectories of individual photons as random walkers that move through a material by sampling both the step size and the direction from distributions that are calculated using the material parameters. 

##### Outline of  procedure

1. A user-specified number of photon packets is created with an initial position, direction of propagation, and weight.
2. The photon packets travel a step size sampled from a distribution, and the positions are updated. 
3. The photon packet weights are updated to account for the light absorbed during the step.
4. The photon packets are scattered after traveling the initial step, and new direcitons are sampled from a distribution.
5. The step size is then sampled again and the process of scattering and stepping is repeated many times as specified by the user.
6. After the simulation is completed, a reflectance is calculated by counting the photon packets that exit the sample in the backscattering direction. 


##### Sampling the direction

The direction at each step is sampled from the phase function, which is the angular distribution of the scattered intensity. The phase function $p$ is:

$$ p = \frac{dC_{sca}}{d\Omega} \frac{1}{C_{sca}} $$ 

where $\frac{dC_{sca}}{d\Omega} $ is the differential scattering cross section and $C_{sca}$ is the total scattering cross section. $C_{sca}$ is calculated for a disordered assembly of particles using a single-scattering model$^{1}$ that takes into account both the form factor and structure factor of the system. The form factor comes from the Mie theory and accounts for the light scattered from individual particles$^{2}$. The structure factor comes from the spatial arrangement of the particles, and accounts for the constructive interference of the scattered light$^{2}$.

##### Sampling the step size

The step size $s$ for each step is given by$^3$:

$$ s = \frac{ln(1-\zeta)}{\mu_T}$$

where $\zeta$ is a random number uniformly distributed from 0 to 1 and $\mu_T$ is the total extinction coefficient. The step size expression comes from the Beer-Lambert law, which states that light is attenuated exponentially as is propagates through a material, and the parameters of that exponential depend on the properties of the material.

##### Calculating Absorption

Each photon packet is assigned an initial weight of 1. As the photon packet travels through the sample medium, some of the light in the packet is absorbed according to the Beer-Lambert law, and the weights $w$ of the packets are absorbed according to:

$$ w = \exp{-\frac{l_{path}}{l_{abs}}} $$

where $l_{path}$ is the total distance travelled by the photon packet, and $l_{abs}$ is the absorption length of the sample medium. With each step that a photon packet takes, the $l_{path}$ increases, causing the weight to decrease.

##### Calculating Reflectance

Each simulation sends in a number of photon packets set by the user. To calculate the reflectance, the number of photon packets that exits the sample in the backscattering direction is divided by the total number of photon packets sent into the sample.

A sample consists of a film of a set thickness. The code also accounts for reflections due to the index contrast at the interface using the Fresnel equations, but only at the first air-sample interface. Refraction of the backscattered light exiting the sample is calculated using Snell's law, so that the angle of photons exiting the sample can be tracked in case the user is only interested in collecting a certain angle range of light. 

To account for the light which is not reflected due to absorption, the weight for each photon at the time of reflectance is multiplied by the reflectance contribution of that photon.

##### References

1. S. Magkiriadou, J. G. (2014). “On the Absence of Red Structural Color in Photonic Glasses, Bird Feathers, and Certain Beetles.” . Phys. Rev. E, 90, 062302.
2. C. F. Bohren, D. R. (1983). Absorption and Scattering of Light by Small Particles. . New York: Wiley.
3. L. Wang, S. L. (1995). MCML - Monte Carlo modeling of light transport in multi-layered tissues. Computer Methods and Programs in Biomedicine , 47, 131-146.

#### Basic generative model

The goal of the generative model is to calculate reflection spectra from structural color samples defined by a set of parameters. The Monte Carlo model described above generates theoretical spectra but these spectra do not correspond to experimental measurements. The full generative model incorporates the Monte Carlo calculations as well as additional factors to produce reflection spectra that resemble experimental data.

All of the arguments taken by the Monte Carlo model can be measured for typical samples, except for particle volume fraction $\phi$, which is included in the generative model as an inference parameter. The Monte Carlo model returns a theoretical reflection calculation $R^t(\lambda, \phi)$, where $\lambda$ is the light wavelengths at which reflectance is calculated. A spectrum $\left \{ R^t(\phi) \right \}$ consists of a set of $R^t_i(\lambda_i, \phi)$ values, where $\left \{ \lambda_i \right \}$ represents all wavelengths used to produce the spectrum.

The theoretical reflectance $R^t$ must be transformed into a corrected reflectance $R^c$ before it can be compared with experimental data. Not all of the incident light on a non-absoribing sample is recorded as being reflected or transmitted, although this must be true. Some light is lost in the experimental, likely due to reflections off coverslip  interfaces, escape from the integrating sphere, or absorption within the system. Since the exact reasons for loss are not known, they cannot be accurately modeled. Instead, We assume only that there is some loss in intensity $L$, and that the loss is not constant at different wavelengths. Therefore a corrected spectrum $\left \{ R^c(\phi) \right \}$ is given by the elementwise product of $\left \{ R^c(\phi) \right \}$ and $\left \{ L \right \}$, or:

$R^c_i=L_iR^t_i(\lambda_i, \phi)$

The $L_i$ values are not generally known (or physically interesting), so they are marginalized over in the inference calculation (performed with MCMC so marginalization is trivial).

#### Generative model extensions

There are multiple ways to extend the basic model described above. All require modification of the underlying Monte Carlo model.

The highest priority extension is to calculate transmission spectra $\left \{ T^c(\phi) \right \}$ as well as reflection spectra. Both spectra can be measured experimentally from the same sample, so this extension would increase the amount of information that the generative model can output. By providing more information to inform our likelihood function, we hope to obtain a better estimate of sample volume fraction $\phi$. In general, there are some losses associated with both transmission and reflection, and there is no obvious simple relation between these values. Separate loss parameters $\left \{ L_R \right \}$ and $\left \{ L_T \right \}$ will be introduced at each wavelength and marginalized over in the inference calculation. This addition would require large changes to the Monte Carlo code, but few changes to the generative model, and is expected to dramatically increase the usefulness of the inference calculation.

Other extensions involve increasing the amount of information that goes into the Monte Carlo calculation by allowing parameters to vary that are currently held fixed. Examples include sphere polydispersity and incident beam divergence, both of which are currently fixed at zero. Both values could be measured experimentally or sampled as parameters and marginalized over in an inference calculation. These types of extensions would help to make the generative model more physical by accounting for phenomena that are currently neglected. They would require relatively minor changes to the Monte Carlo code, but the generative model may need to be reworked for computation time concerns. The basic model includes only a single parameter ($\phi$) that affects the Monte Carlo output, but addition of more parameters would increase the dimensionality and computation time required to adequately sample parameter space.

#### Model uncertainties


#### Likelihood Function

Due to computation time concerns, we hope to calculate our likelihood in two steps. Walkers will explore a posterior distribution in the parameter space defined by the loss values between each step they take in the volume fraction dimension. For this intermediate likelihood calculation, a theoretical spectrum $\left \{ R^t(\phi) \right \}$ has already been calculated. There are uncertainties associated with both the stochastic Monte Carlo model and the experimental data. These two uncertainties can be combined into a likelihood as in Gregory eq. 4.51. 

In [2]:
def likelihood(spect_theory, loss, spect_data, model_sigma, meas_sigma):
    """
    returns likelihood
    
    Parameters:
        spect_theory: a spectrum calculated from the MC model (array of length N) 
        loss: reflection loss parameters (array of length N)
        wavelength: independent variable (array of length N)
        relection: dependent variable - experimental measurements (array of length N)
        model_sigma: uncertainty associated with the probabilistic MC model (array of length N)
        meas_sigma: uncertainty assocated with the experimental measurements (array of length N)
    """
    
    
    residual = (spect_data - (1-loss)*spect_theory)
    var_eff = model_sigma**2 + meas_sigma**2
    chi_square = np.sum(residual**2/var_eff)
    prefactor = 1/np.prod(np.sqrt(2*np.pi*var_eff))
    return prefactor * np.exp(-chi_square/2)